In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable


In [2]:
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 10
learning_rate = 0.001

In [3]:
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),  
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
transform=transforms.ToTensor())

In [4]:
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
shuffle=False)

In [5]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()
        self.main= nn.Sequential(nn.Linear(input_size, hidden_size), 
                                 nn.ReLU(),
                                 nn.Linear(hidden_size, hidden_size),
                                 nn.ReLU(),
                                 nn.Linear(hidden_size, num_classes),
                                nn.Sigmoid())
    
    def weight_init(self,):
        for m in self._modules:
            normal_init(self._modules[m],)
            
    
    
    def forward(self, x):
        inp=self.main(x)
        return inp

In [6]:
net = Net(input_size, hidden_size, num_classes)

In [7]:
def normal_init(m):
    if isinstance(m,nn.Linear):
        
        m.weight.data.xavier_normal()
        m.bias.data.zero_()

In [8]:
net=net.cuda()
net.weight_init()

print (net)

Net (
  (main): Sequential (
    (0): Linear (784 -> 500)
    (1): ReLU ()
    (2): Linear (500 -> 500)
    (3): ReLU ()
    (4): Linear (500 -> 10)
    (5): Sigmoid ()
  )
)


In [9]:
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)


In [10]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Convert torch tensor to Variable
        images = Variable(images.view(-1, 28*28).cuda())
        labels = Variable(labels.cuda())
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
%(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch [1/5], Step [100/6000], Loss: 1.6958
Epoch [1/5], Step [200/6000], Loss: 1.5919
Epoch [1/5], Step [300/6000], Loss: 1.6094
Epoch [1/5], Step [400/6000], Loss: 1.6068
Epoch [1/5], Step [500/6000], Loss: 1.5896
Epoch [1/5], Step [600/6000], Loss: 1.5781
Epoch [1/5], Step [700/6000], Loss: 1.5627
Epoch [1/5], Step [800/6000], Loss: 1.5361
Epoch [1/5], Step [900/6000], Loss: 1.4988
Epoch [1/5], Step [1000/6000], Loss: 1.5454
Epoch [1/5], Step [1100/6000], Loss: 1.4626
Epoch [1/5], Step [1200/6000], Loss: 1.7011
Epoch [1/5], Step [1300/6000], Loss: 1.5286
Epoch [1/5], Step [1400/6000], Loss: 1.4946
Epoch [1/5], Step [1500/6000], Loss: 1.5706
Epoch [1/5], Step [1600/6000], Loss: 1.7224
Epoch [1/5], Step [1700/6000], Loss: 1.5577
Epoch [1/5], Step [1800/6000], Loss: 1.5671
Epoch [1/5], Step [1900/6000], Loss: 1.5738
Epoch [1/5], Step [2000/6000], Loss: 1.4674
Epoch [1/5], Step [2100/6000], Loss: 1.4632
Epoch [1/5], Step [2200/6000], Loss: 1.4908
Epoch [1/5], Step [2300/6000], Loss: 1.46

Epoch [4/5], Step [800/6000], Loss: 1.4616
Epoch [4/5], Step [900/6000], Loss: 1.4885
Epoch [4/5], Step [1000/6000], Loss: 1.4885
Epoch [4/5], Step [1100/6000], Loss: 1.4997
Epoch [4/5], Step [1200/6000], Loss: 1.5141
Epoch [4/5], Step [1300/6000], Loss: 1.5612
Epoch [4/5], Step [1400/6000], Loss: 1.4885
Epoch [4/5], Step [1500/6000], Loss: 1.4748
Epoch [4/5], Step [1600/6000], Loss: 1.4612
Epoch [4/5], Step [1700/6000], Loss: 1.5751
Epoch [4/5], Step [1800/6000], Loss: 1.5150
Epoch [4/5], Step [1900/6000], Loss: 1.6589
Epoch [4/5], Step [2000/6000], Loss: 1.5748
Epoch [4/5], Step [2100/6000], Loss: 1.4748
Epoch [4/5], Step [2200/6000], Loss: 1.4777
Epoch [4/5], Step [2300/6000], Loss: 1.4763
Epoch [4/5], Step [2400/6000], Loss: 1.4746
Epoch [4/5], Step [2500/6000], Loss: 1.4612
Epoch [4/5], Step [2600/6000], Loss: 1.4612
Epoch [4/5], Step [2700/6000], Loss: 1.5748
Epoch [4/5], Step [2800/6000], Loss: 1.4887
Epoch [4/5], Step [2900/6000], Loss: 1.4885
Epoch [4/5], Step [3000/6000], Los

In [11]:
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28).cuda())
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 92 %
